Hikayemiz: Bir Sınıftaki Öğrencilerin Fısıltı Oyunu 🤫
Haydi, bir sınıf hayal edelim. Bu sınıftaki her bir öğrenci, bir cümlenin içindeki bir kelimeyi temsil ediyor.

Örneğin batch_size = 4: Sınıfta 4 tane birbirinden bağımsız "oyun grubu" var (batch).block_size = 8: Her grupta 8 öğrenci (kelime) var. Bu öğrenciler sırayla oturuyorlar. Cümlenin sırası gibi.
n_embd = 64: Her öğrencinin kendine ait 64 farklı "özelliği" var. Mesela bir öğrencinin "neşeli", "çalışkan", "yorgun" gibi 64 farklı özelliği olabilir. Bu, kelimenin anlamını taşıyan bir vektör.
Kodumuzdaki x değişkeni, işte bu 4 grubun, 8'er öğrenciden oluşan ve her öğrencinin 64 özelliği olan sınıf listesidir.

--------------

Adım 1: Herkesin Üç Farklı Şapka Takması (Key, Query, Value)
Sınıfta bir oyun oynayacağız. Oyunun adı "Anlamı Zenginleştirme". Her öğrencinin (kelimenin) diğer öğrencilere bakarak kendi anlamını daha iyi kavraması gerekiyor.

Bunun için her öğrenciye üç farklı "rol" veya "şapka" veriyoruz. Bu şapkaları takınca, sahip oldukları 64 özellik, daha küçük ve amaca yönelik 4 özelliğe dönüşüyor (head_size = 4).

-----------------
Sorgu Şapkası (Query ❓): Bir öğrenci bu şapkayı taktığında, diğerlerine sormak istediği soruyu hazırlar. Bu, "Ben kiminle ilgiliyim? Bana kimin anlamı lazım?" diye bağırmak gibidir.
q = query(x) -> Her öğrenci sorgu şapkasını taktı ve elinde bir "soru kartı" (q) oluştu.

Anahtar Şapkası (Key 🔑): Bir öğrenci bu şapkayı taktığında, kendisinin ne hakkında olduğunu özetleyen bir "anahtar kelime" veya "etiket" oluşturur. Bu, "Benim konum bu! Eğer benimle ilgili bir şey arıyorsan, bu etikete bak!" demek gibidir.
k = key(x) -> Her öğrenci anahtar şapkasını taktı ve boynuna bir "etiket" (k) astı.

Değer Şapkası (Value 🎁): Bu şapkayı takan öğrenci ise, eğer birisi ona ilgi gösterirse sunacağı "bilgi paketini" hazırlar. Bu, "Eğer bana dikkat etmeye karar verirsen, sana vereceğim değerli bilgi bu." demek gibidir.
v = value(x) -> Her öğrenci değer şapkasını taktı ve elinde bir "hediye paketi" (v) tutuyor.

Her öğrenci, elindeki "soru kartını" (q) kaldırır ve sınıftaki diğer tüm öğrencilerin boynundaki "etiketlere" (k) bakar.

q @ k.transpose(...): Bu matematiksel işlem (@), bir matris çarpımıdır ama biz buna "ilgi ölçer" diyelim. Her öğrencinin sorusu, diğer tüm öğrencilerin etiketleriyle "çarpılır" ve bir uyumluluk puanı ortaya çıkar.
Eğer bir öğrencinin sorusu, başka bir öğrencinin etiketiyle çok uyumluysa, aralarındaki puan (wei, yani weight/ağırlık) yüksek olur. Uyumsuzsa, puan düşük olur.
Sonuçta elimizde ne var? 8x8'lik bir tablo! Bu tablo, her öğrencinin diğer her bir öğrenciye (kendisi de dahil) ne kadar ilgi duyduğunu gösteren bir "ilgi haritası"dır.
Adım 3: Geleceği Görmek Yasak! (Maskeleme)
Bu oyunda önemli bir kural var: Öğrenciler kendilerinden sonra gelen öğrencilerden kopya çekemez! Yani bir kelime, cümlenin ilerisindeki bir kelimenin anlamından etkilenemez. Sadece kendinden öncekilere bakabilir.

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
torch.tril(...): Bu kod, sihirli bir "kural maskesi" (tril) yaratır. Bu maske, ilgi haritamızla aynı boyutta (8x8) ve sadece sol alt üçgeni 1'lerle dolu. Geri kalanı 0. (1 = bakabilirsin, 0 = bakamazsın).
masked_fill(...): Bu komutla kuralı uyguluyoruz. İlgi haritamızdaki (wei) bakılması yasak olan tüm yerleri (maskenin 0 olduğu yerleri) negatif sonsuz (-inf) puanı ile doldururuz. Bu, "Oraya bakman o kadar yasak ki, ihtimali bile yok!" demektir.
Artık ilgi haritamız, her öğrencinin sadece kendisi ve kendinden öncekilere olan ilgi puanlarını içeriyor.

Adım 4: İlgi Dağılımı (Softmax)
Elimizde ham ilgi puanları var. Ama bir öğrencinin toplamda %100'lük bir "dikkat" kaynağı var. Bu kaynağı, ilgi duyduğu kişilere nasıl dağıtacak? İşte burada Softmax devreye giriyor.

wei = F.softmax(wei, dim=-1)
Softmax, ham ve anlamsız puanları (örneğin: 2.3, -1.5, 10.8) alır ve onları güzel bir olasılık dağılımına dönüştürür. Yani, her öğrenci için ilgi duyduğu diğer öğrencilere verdiği dikkat yüzdelerini hesaplar.

Puanı yüksek olan öğrenci, dikkatin büyük bir kısmını alır (%70 gibi).
Puanı düşük olan öğrenci, az bir dikkat alır (%5 gibi).
Puanı negatif sonsuz olan (gelecekteki öğrenciler) ise %0 dikkat alır.
Sonuçta wei tablomuz, artık her öğrencinin dikkatini geçmişteki hangi öğrencilere yüzde kaç oranında dağıttığını gösteren net bir "dikkat haritası" haline geldi.

Adım 5: Bilgileri Toplama ve Anlamı Zenginleştirme
Geldik son adıma! Artık her öğrenci kimlere ne kadar dikkat edeceğini biliyor. Şimdi, o dikkat oranında onlardan bilgi toplama zamanı.

out = wei @ v
Her öğrenci, elindeki "dikkat haritasına" (wei) bakar.
Diyelim ki 3. öğrenci, 1. öğrenciye %20, 2. öğrenciye ise %80 dikkat etmeye karar verdi.
O zaman 1. öğrencinin elindeki "hediye paketinden" (v) %20 alır, 2. öğrencinin hediye paketinden ise %80 alır.
Bu topladığı bilgileri birleştirerek kendi yeni zenginleştirilmiş bilgisini (out) oluşturur.
İşte bu out değişkeni, her öğrencinin (kelimenin) sadece kendi başına değil, cümlenin bağlamına (kendinden önceki kelimelere) dikkat ederek oluşturduğu yeni ve daha derin anlamıdır!

#1.Bölüm:Projenin Planı ve Malzemeler(Hyperparameters & Data)

##A.Kütüphaneleri dahil etme

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

##B.Hiperparametreler

In [2]:
# hyperparameters
batch_size = 16 # Paralel olarak kaç tane bağımsız metin dizisini işleyeceğiz?
block_size = 32 # Tahminler için maximum bağlam uzunluğu (Ne kadar geçmişe bakabilir?)
max_iters = 5000 # Toplamda kaç eğitim adımı(ders) yapılacak?
eval_interval = 100 # Her 100 derste bir öğrenme durumunu kontrol et(sınav yap)
learning_rate = 1e-3 # Öğrenme hızı(Her hatadan sonra ne kadar büyük bir düzeltme yapacak?)
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Dersleri GPU'da mı(hızlı) yoksa CPU'da mı(yavaş) işleyeceğiz?
eval_iters = 200 # Sınavda kaç soru sorulacak?
n_embd = 64 # Her bir karakterin/kelimenin anlamını kaç sayıyla temsil edeceğiz?(Anlam Zenginliği)
n_head = 4 # Kaç tane 'dikkat başlığı' (farklı bakış açısı) olacak
n_layer = 4 # Kaç tane 'transformatör bloğu' (düşünme katmanı) üst üste konulacak?
dropout = 0.0 # Ezberlemeyi önlemek için ne kadar 'unutkanlık' ekleyeceğiz ?(% 0, yani şimdilik hiç)

In [3]:
torch.manual_seed(42)

##C.Veri Setini Yükleme

In [4]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2025-06-17 19:16:51--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.01s   

2025-06-17 19:16:51 (83.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



##D.Karakter Seti

In [5]:
# Metindeki tüm benzersiz karakterleri buluyoruz.Bu bizim 'alfabemiz'
chars = sorted(list(set(text)))
vocab_size = len(chars) # Alfabemizde kaç karakter var ?

# Karakterileri sayılara (stoi) ve sayıları karakterlere çeviren (itos) sözlükler oluşturuyoruz
# Çünkü bilgisayar harflerle değil sayılarla çalışır
stoi = {ch : i for i , ch in enumerate(chars)}
itos = {i : ch for i , ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]        # Bir metni sayı dizisine çevirir
decode = lambda l : ''.join([itos[i] for i in l])     # Bir sayı dizisini metne çevirir


# Veriyi Eğitim ve Sınav(Validasyon) olarak ikiye ayırıyoruz

data = torch.tensor(encode(text) , dtype = torch.long)
n = int(0.9*len(data))   # Verinin %90 'ı eğitim , % 10 sınav için
train_data = data[:n]
val_data= data[n:]



#2.Bölüm : Ders ve Sınav Hazırlığı

##A.Veri Yükleme / Ders Hazırlama Fonksiyonu

In [6]:
# bu fonksiyon her ders için metinden rastgele parça seçer

def get_batch(split):
  # Eğitim veya sınav setinden veri seçilir
  data = train_data if split == 'train' else val_data
  # Metin içinden rastgele başlangıç noktaları seçilir (batch_size kadar)
  ix = torch.randint(len(data) - block_size,(batch_size,))
  # Seçilen noktalardan block_size uzunluğunda metin parçaları(x) alınır
  x = torch.stack([data[i : i + block_size] for i in ix])
  # Hedefler (y) , x 'in bir sonraki karakteridir.Modelden bunu tahmin etmesini isteyeceğiz.
  y = torch.stack([data[i+1: i + block_size+1] for i in ix])
  return x , y

  #Hikayesi: Öğrenciye bir metin parçası (x) veriyoruz ve "Bu metindeki her bir karakterden sonra hangi karakter gelir?" diye soruyoruz.
  # Doğru cevaplar da y oluyor.

##B.Sınav Yapma Fonksiyonu - Kayıp Tahmin




In [7]:
# Bu fonksiyon , modelin ne kadar iyi öğrendiğini ölçer.

@torch.no_grad()  # Bu blokta öğrenme(gradyan hesaplama) yapma , sadece değerlendir demek için dekoratörü kullandık.

def estimate_loss():
  out = {}
  model.eval() # modeli sınav moduna al

  for split in ['train','val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X , Y = get_batch(split) # Bir derslik veri al
      logits , loss = model(X ,Y) # Modelin tahminini ve hatasını al
      losses[k] = loss.item()     # Hatam miktarını kaydet
    out[split] = losses.mean()    # Ortalama hatayı hesapla
  model.train()   # Modeli tekrar "Öğrenme Moduna" al
  return out

#Hikayesi: Modelin öğrenmesini geçici olarak durdururuz.
#Ona bir sürü soru (eval_iters kadar) sorarız ve ortalama ne kadar hata yaptığına bakarız.
#Bu, ezber mi yapıyor yoksa gerçekten öğreniyor mu anlamamızı sağlar.

#3.Bölüm : Modelin İnşaası (Beynin Parçaları)
- Şimdi modelin kendisini, yani "dijital beyni" inşa ediyoruz. Bu beyin, katman katman çalışan parçalardan oluşur.



##A.Head Sınıfı : Tek bir Dikkat Kafası
- Bir kelimenin/karakterin, kendinden öncekilere nasıl dikkat edeceğini belirler.

In [8]:
class Head(nn.Module):
  def __init__(self,head_size):
    super().__init__()

    # Query , Key , Value katmanları : Her kelimeye/karaktere üç farklı şapka takar

    self.key = nn.Linear(n_embd , head_size , bias = False)
    self.query = nn.Linear(n_embd , head_size , bias = False)
    self.value = nn.Linear(n_embd , head_size , bias = False)

    # Geleceğe bakmayı engelleyen maske --- Not : İlişkileri sadece geçmişe bakarak öğrenecek.Geleceğe bakmak yok
    # Satır ve sütun sayısı block_size olan iki boyutlu alt taraf diagonal 1 olan bir matris oluştur : İlk satırdaki eleman sadece kendini görür-2.Satırdaki eleman bir öncekini ve kendini görür...
    self.register_buffer('tril' , torch.tril(torch.ones(block_size,block_size)))
    # Unutma oranı
    self.dropout = nn.Dropout(dropout)

    #Ne oluyor? Tek bir self-attention başı tanımlıyoruz. Her biri, giriş vektörlerini (n_embd=64) daha küçük bir boyuta (head_size=16) dönüştürüyor (key, query, value için).
    # tril, gelecekteki zaman dilimlerini maskelemek için alt üçgen matris. Dropout, overfitting’i önler (ama dropout=0).
    #Hikaye bağlamı: Öğrencimiz, her kelimeyi/karakteri 64 boyutlu bir notla temsil ediyor ve bu notları n_embd // n_head =  16 boyutlu özetlere (key, query, value) indirgiyor. Geleceği görmemek için bir perde (tril) kullanıyor.

  def forward(self,x):
    B , T , C = x.shape
    k = self.key(x)  # (B , T , C)
    q = self.query(x)  # (B, T , C)
    # Dikkat skorlarını hesapla
    wei = q @ k.transpose(-2,-1) * C**-0.5          #B, T, C) @ (B, C, T) -> (B, T, T) ---Not:** C**-0.5 ile ölçekleme, sayıların çok büyümesini engelleyerek eğitimi stabil hale getirir.
    wei = wei.masked_fill(self.tril[:T , :T] == 0 , float('-inf'))  # (B , T , T)
    wei= F.softmax(wei , dim = -1)  # (B , T , T)
    wei = self.dropout(wei)
    # Değerleri (value) dikkat skorlarına göre ağırlıklı topla
    v = self.value(x)  # (B , T , C)
    out = wei @ v   # (B , T , T) @ (B , T , C) ----> (B , T , C)
    return out

**Ne oluyor? Self-attention mekanizması:**
- 1. Giriş x (B,T,64)’ü key, query, value’ye dönüştürüyor (her biri B,T,16).
- 2. Dikkat skorları (wei) hesaplanıyor: query ve key’in nokta çarpımı, normalize edilmiş (C**-0.5 ile ölçekleme, patlamayı önler).
- 3. Gelecek maskeleniyor (tril), softmax ile ağırlıklar normalize ediliyor.
- 4. Value vektörleri ağırlıklı olarak birleştiriliyor.
**Hikaye bağlamı: Öğrencimiz, her harfin diğer harflerle ilişkisini (query-key) ölçüyor, sadece geçmişe bakıyor, ağırlıkları hesaplıyor ve geçmiş harflerin notlarını (value) birleştiriyor.
Boyut: Giriş (B,T,64), çıkış (B,T,16).

##B.MultiHeadAttention Sınıfı --- Çoklu Dikkat,
- Neden tek bir bakış açısıyla yetinelim?
- Bu modül birden fazla Head 'i paralel olarak çalıştırır

In [14]:
class MultiHeadAttention(nn.Module):
  def __init__(self , num_heads , head_size):
    super().__init__()
    # Belirtilen sayıda Head oluştur ve bir listede tut
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    # Head'lerden gelen sonuçları birleştiren bşr projeksiyon katmanı
    self.proj = nn.Linear(n_embd , n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self,x):
    # Her bir Head sonucunu hesapla ve bunları birleştir
    out = torch.cat([h(x) for h in self.heads] , dim = -1 )
    # Birleştirilmiş sonucu projeksiyon katmanından geçir
    out = self.dropout(self.proj(out))
    return out

# Hikayesi: Bir konuyu anlamak için farklı uzmanlara danışmak gibidir.
#Bir Head kelimelerin fiil-nesne ilişkisine odaklanırken, diğeri sıfat-isim ilişkisine odaklanabilir.
#Sonra tüm bu uzmanların görüşlerini (out) birleştirip ortak bir sonuca (proj(out)) varırız.

**Ne oluyor?**
-  n_head=4 tane dikkat başını paralel çalıştırıyoruz. Her baş, head_size=16 (çünkü n_embd=64 / n_head=16). Sonuçlar birleştirilip proj ile tekrar n_embd=64 boyutuna getiriliyor.

**Hikaye bağlamı:**
-  Öğrencimiz, 4 farklı açıdan metni inceletiyor. Sonra hepsinin notlarını birleştirip son bir özet yazıyor.
-  Her başın çıktısı (B,T,16) birleştiriliyor (B,T,64), sonra lineer dönüşüm ve dropout uygulanıyor.
Boyut: Giriş (B,T,64), çıkış (B,T,64).

##C. FeedForward Sınıfı : Düşünme Modülü
- Dikkat mekanizması kelimelerin birbiriyle iletişim kurmasını sağlar.
- Bu katman ise her kelimenin topladığı bilgilerle tek başına düşünmesini sağlar.

In [10]:
class FeedForward(nn.Module):
  def __init__(self , n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd , 4 * n_embd), #Gelen bilgiyi daha geniş bir alana yay --> Daha derin düşünmek için
        nn.ReLU(),                      # Negatif bilgileri at , sadece pozitif bilgileri tut
        nn.Linear(4 * n_embd , n_embd), # Düşünceyi tekrar orjinal boyutuna sıkıştır
        nn.Dropout(dropout),
    )

  def forward(self,x):
    return self.net(x)

#Hikayesi: Öğrenci, diğerlerinden fısıltıyla bilgi topladıktan sonra (MultiHeadAttention),
# sırasına çekilip bu bilgileri kendi içinde işler, analiz eder ve bir sonuca varır.

## D. Block Sınıfı : Bir derslik Ünite
- Bu bir transformatörün temel yapı taşıdır.
- Bir "iletişim" adımını(MultiHeadAttention) ve bir "düşünme" adımını(FeedForward) birleştirir.

In [11]:
class Block(nn.Module):
  def __init__(self , n_embd , n_head):
    super().__init__()
    head_size = n_embd // n_head

    self.sa = MultiHeadAttention(n_head , head_size) # İletişim modülü
    self.ffwd = FeedForward(n_embd)                  # Düşünme modülü
    self.ln1 = nn.LayerNorm(n_embd)                  # Normalizasyon katmanları
    self.ln2  = nn.LayerNorm(n_embd)

  def forward(self,x):
    # Önceki bilgi (x) + yeni iletişimden gelen bilgi (sa). Buna "residual connection" denir
    x = x + self.sa(self.ln1(x))
    # Bir önceki adımın sonucu (x) + yeni düşünme modülünden gelen bilgi(ffwd)
    x = x + self.ffwd(self.ln2(x))
    return x

# Hikayesi : Bir ders ünitesi gibidir.Önce öğrenciler birbiriyle konuşur(sa)
# sonra öğrendiklerini kendi başına düşünürler(ffwd)
# x = x + ... kısmı çok önemlidir : Bu öğrenci, "yeni bir şey öğrenirken eski bildiklerini unutmasın , sadece üzerine eklesin" demektir

# 4. Bölüm : Beynin Tamamlanması ve Çalıştırılması

- Tüm Parçaları birleştirip tam bir dil modeli oluşturuyoruz

## BigramLanguageModel Sınıfı

In [16]:
class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    # Her bir karakterin başlangıç anlamını tutan tablo
    self.token_embedding_table = nn.Embedding(vocab_size , n_embd)
    # Her bir pozisyonun (1. , 2. , 3. sıra ...) anlamını tutan tablo
    self.position_embedding_table = nn.Embedding(block_size , n_embd)
    # Belirtilen sayıda Block'u (ders ünitesini) üst üste koy
    self.blocks = nn.Sequential(*[Block(n_embd , n_head = n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd) # Son bir Normalizasyon
    # Son zenginleştirilmiş anlamı  : alfabedeki her harf için bir skora dönüştüren katman
    self.lm_head = nn.Linear(n_embd , vocab_size)


  def forward(self , idx , targets = None):
    B , T = idx.shape
    # Gelen sayıların (idx) karakter ve pozisyon anlamlarını tablolardan al
    tok_emb = self.token_embedding_table(idx) # (B ,  T , C)
    pos_emb = self.position_embedding_table(torch.arange(T , device = device))  # (T , C)

    #Bir karakterin anlamı  = kendi anlamı  + pozisyonun anlamı
    x = tok_emb + pos_emb  # (B , T , C)
    #Veriyi tüm düşünme katmanlarından geçir
    x = self.blocks(x)  # (B , T , C)
    x = self.ln_f(x)   # (B , T , C)
    # Sonucu bir sonraki karakter için tahmin skorlarına (logits) dönüştür
    logits = self.lm_head(x)  # (B , T , vocab_size)

    # Eğer hedef(doğru cevap) verilmişse , hatayı (loss) hesapla
    if targets is None:
      loss = None
    else :
      B , T , C = logits.shape
      logits = logits.view(B * T , C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits , targets)
    return logits , loss


  def generate(self , idx , max_new_tokens):
    # Bu fonksiyon yeni metin üretir
    for _ in range(max_new_tokens):
      # Bağlamı son block_size kadar kırp
      idx_cond = idx[: , -block_size:]
      #Bir sonraki karakter için tahminleri al
      logits , loss = self(idx_cond)
      # Sadece son karakterin tahminine odaklan
      logits = logits[: , -1 , :] # Yeni şekil : (B , C)
      # Skorları olasılıklara çevir
      probs = F.softmax(logits , dim = -1) #  (B , C)
      # Bu olasıklara göre rastgele bir sonraki karakteri seç
      idx_next = torch.multinomial(probs , num_samples = 1) # (B , 1)
      # Seçilen karakterleri mevcut metne ekle
      idx = torch.cat((idx , idx_next) , dim = 1) # (B , T + 1)
    return idx

#5.Bölüm : Eğitim Süreci(Okul Günü)
- Modeli Yarattık  , şimdi onu eğitme zamanı

In [17]:
model = BigramLanguageModel()
m = model.to(device)   # modeli GPU'ya taşı

# Bİr optimize edici oluştur.Bu modelin hatalarından ders almasını sağlar
optimizer = torch.optim.AdamW(model.parameters() , lr = learning_rate)

# Eğitim döngüsü
for iter in range(max_iters):
  # Arada bir sınav yap ve sonuçları yazdır
  if iter % eval_interval == 0:
    losses = estimate_loss()
    print(f"step {iter} : train loss {losses['train'] : .4f} , val loss {losses['val'] : .4f}")


  #Bir derslik veri al (bir 'batch')
  xb , yb = get_batch('train')

  # modeli çalıştır ve hatayı hesapla
  logits , loss = model(xb , yb)
  optimizer.zero_grad(set_to_none = True) #Eski hataları temizle
  loss.backward()                         # Yeni hatalardan kimin ne kadar sorumlu olduğunu hesapla
  optimizer.step()              #Parametreleri (ağırlıkları) güncelle , yani öğrenmeyi gerçekleştir


# Hikayesi : Bu döngü modelin "okul günüdür".
# Sürekli olarak dersler (get_batch) alır , ne kadar hata yaptığını görür(loss).
# Bu hatalardan ders çıkarır(loss.backward())
# Ve kendini geliştirir(optimizer.step())
# Arada sırada da sınava girer (estimate_loss)

step 0 : train loss  4.2601 , val loss  4.2646
step 100 : train loss  2.6407 , val loss  2.6446
step 200 : train loss  2.4995 , val loss  2.5005
step 300 : train loss  2.4084 , val loss  2.4203
step 400 : train loss  2.3502 , val loss  2.3623
step 500 : train loss  2.3078 , val loss  2.3188
step 600 : train loss  2.2527 , val loss  2.2872
step 700 : train loss  2.2054 , val loss  2.2191
step 800 : train loss  2.1721 , val loss  2.1978
step 900 : train loss  2.1336 , val loss  2.1531
step 1000 : train loss  2.1000 , val loss  2.1433
step 1100 : train loss  2.0578 , val loss  2.1181
step 1200 : train loss  2.0477 , val loss  2.1043
step 1300 : train loss  2.0169 , val loss  2.0757
step 1400 : train loss  2.0027 , val loss  2.0602
step 1500 : train loss  1.9665 , val loss  2.0372
step 1600 : train loss  1.9371 , val loss  2.0201
step 1700 : train loss  1.9286 , val loss  2.0081
step 1800 : train loss  1.9166 , val loss  2.0134
step 1900 : train loss  1.8929 , val loss  1.9894
step 2000 : 

# 6.Bölüm : Mezuniyet -- Metin Üretme
- Eğitim bitti.
- Bakalım Ne Öğrendi

In [18]:
# Başlangıç olarak boş bir bağlam (0) veriyoruz
context = torch.zeros((1,1) , dtype = torch.long , device = device)
#Modelden 2000 yeni karakter üretmesini istiyoruz ve sonucu metne çevirip yazdırıyoruz
print(decode(m.generate(context,max_new_tokens=2000)[0].tolist()))

#Hikayesi: Modelimize boş bir sayfa veriyoruz ve "Hadi,bildiklerinle bir şeyler yaz!" diyoruz.
# O da karakter karakter , öğrendiği metin tarzında yeni bir metin oluşturuyor.


But make to never. May, whose this not.

Ly blear; I would there fance more: I wound cut be the divist,
Wring arcy mistine the good, stout,
And bluds, If both your poicy.

ESCALUS Cloming,
O wimpon, gentle of ma'gam!
'Twout thatIe thy donot; there go dovery;
TheIe be that what calland they arm
That you thou abed to be jurtuants,
Ow speak the vice no-moler of thousand Look loves,
Give which tread well all them guards
Inlins thus rown thing us; that for I land?
It no be doth nor thou horsight holO
Yet dive though long, prisons of,
Got to be see, Clencuar; why, eor that
sees I' dare farew, and, earful:
'We Eve due revent to his so the suire adverys broatted lies,
Then ailf Oreed ands in my luce,
Deed Your falling his banight from the thesic aring.

KING EDWARV:
Doue he hath wabs them see for Edward,
Me'ly ass' privostantay.s 'Whal is the brack and be
At everments the hath doth I'll upon the sonce?

LUCIO:
Is mist, abutladal gentle the queen hif is oneBabestal,
Plan day I dore.
That I now